# Homework 4: SQL, FEC Data, and Small Donors

## Due: 11:59pm Tuesday, March 6


In this homework, we explore the money exchanged during the 2016 election using the [Federal Election
Commission's public records](http://www.fec.gov/finance/disclosure/ftpdet.shtml). 

In [ ]:
import pandas as pd
import numpy as np
import re
import sqlalchemy

## Getting Started

For this assignment, you're going to use a collection of Postgre SQL servers hosted on the Berkeley DataHub cluster.  These are shared resources so please try to avoid asking complex queries that might slow down the databases for other students. 

### Connecting to the Shared Postgres Databases

The following block of code will download the list of available database servers on the datahub cluster:

To load balance everyone we will randomly select one of the above databases for this notebook. 

**Note:**  If you run into issues with your current database re-run the following cell until you get a different database.

In [ ]:
def get_db_urls(password="dataisgreat"):
    # Download the (potentially changing) list of databases
    db_list = pd.read_csv("http://ds100.org/sp18/assets/sql/db_list.csv")
    db_list['uri'] = (
        "postgres+psycopg2://" +
        db_list['username'] + ":" +
        password + "@" +
        db_list['host'] + "/" +
        db_list['dbname']
    )
    return db_list

print("Datbase URIs:")
db_list = get_db_urls()
for uri in db_list['uri']:
    print("\t", uri)

In [ ]:
if 'db_url' in globals():
    db_url = db_list[db_list['uri'] != db_url].sample(1).iloc[0]['uri']
else:
    db_url = db_list.sample(1).iloc[0]['uri']
db_url

The following cell will connect to the above database using the SQL Alchemy library.

In [ ]:
engine = sqlalchemy.create_engine(db_url, connect_args={'sslmode':'require'})
connection = engine.connect()

To check that everything is working we will list the available tables.

In [ ]:
engine.table_names()

The following examines the schema of the `indiv_sample_berkeley` table. This table is constructed as SELECT * FROM individual WHERE city = 'BERKELEY'. It contains all the contributions from Berkeley.

In [ ]:
inspector = sqlalchemy.inspect(engine)
pd.DataFrame(inspector.get_columns("indiv_sample_berkeley")).set_index('name')

### Running a Local Postgres Database (Advanced!)

You can also try to setup your own local Postgres Database. You will need:

1. The FEC (and lecture) [SQL data](https://drive.google.com/open?id=1EahF0Le_4Gk6JG0CWzmgkotrepHz0oPf)
1. Install postres [mac](https://postgresapp.com/), [windows](https://www.postgresql.org/download/windows/), or [linux](http://lmgtfy.com/?q=Install+PostgreSQL+on+Linux).
1. Install the FEC data into a Postgres database ([instructions](https://www.postgresql.org/docs/9.1/static/backup-dump.html)).  On mac and linux it looks like:

```bash 
bunzip2 -c filename.gz | psql ds100
```

To connect to your local database you create an engine with the following command:

```python
engine = sqlalchemy.create_engine("postgres://postgres:@localhost:5432/ds100")
```

**Warning:** *For grading purposes it is important that when you submit the notebook it runs correctly using our Postgres servers.


In [ ]:
## Uncomment at your own risk!
# import sqlalchemy
# engine = sqlalchemy.create_engine("postgres://postgres:@localhost:5432/ds100")

## Table Descriptions

Here is a list of the tables in the database.  Each table links to the documentation on the [FEC page](http://www.fec.gov/finance/disclosure/ftpdet.shtml) for the dataset.

Note that the table names here are slightly different from the ones in lecture. Consult the FEC page
for the descriptions of the tables to find out what the correspondence is.

- [`cand`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryCandidateMaster.shtml): Candidates table. Contains names and party affiliation.
- [`comm`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryCommitteeMaster.shtml): Committees table. Contains committee names and types.
- [`indiv_sample_berkeley`](http://www.fec.gov/finance/disclosure/metadata/DataDictionaryContributionsbyIndividuals.shtml): All individual contributions from Berkeley.


In [ ]:
# We use `LIMIT 5` to avoid displaying a huge table.
# Although our tables shouldn't get too large to display,
# this is generally good practice when working in the
# notebook environment.  Jupyter notebooks don't handle
# very large outputs well. 
query = """
SELECT * from indiv_sample_berkeley LIMIT 5
"""
pd.read_sql(query, engine)

In [ ]:
query = '''
SELECT cand_id, cand_name
FROM cand
WHERE cand_pty_affiliation = 'DEM'
LIMIT 5
'''
pd.read_sql(query,engine)

### Notes on Grading 

1. For this homework, we will be grading your SQL expressions so be sure to do all computation in SQL (unless otherwise requested).  
1. Use the query string variable we provide since we will be analyzing your SQL query strings.

### Question 1a

We are interested in finding the individuals from Berkeley that donated large sums. To begin to answer this question, we will look at the `indiv_sample_berkeley` table. We'll find all the transactions that exceed \$100,000. However, if there are a lot of transactions like that, it might not be useful to list them all.  So before actually finding the transactions, find out how many such transactions there are. Use only SQL to compute the answer.

(It should be a table with a single column called **`count`** and a single entry, the number of transactions.)  

We will be grading the query string `query_q1a`.  You may modify our template but the result should contain the same information with the same names.

**In this question, and in the following ones, we will only be grading the output of** `pd.read_sql(query_qXX, engine)`.  
**Make sure that the output of this query matches what you want (order, column names, etc...)**

In [ ]:
# complete the query string
query_q1a = """
SELECT ...
FROM ...
WHERE ...
"""

### BEGIN SOLUTION
query_q1a = '''
SELECT count(*)
FROM indiv_sample_berkeley
WHERE transaction_amt > 100000
'''

### END SOLUTION
pd.read_sql(query_q1a, engine)

In [ ]:
res = pd.read_sql(query_q1a,engine)
assert len(res) == 1
assert res.columns == ['count']

### BEGIN HIDDEN TESTS
assert res["count"][0] == 5
### END HIDDEN TESTS

### Question 1b
Having seen that there aren't too many transactions that exceed \$100,000, let's find them all.  Using only SQL, construct a table containing the committee ID, contributor's name, and the transaction amount, for transactions that exceed $100,000 dollars.  Sort the transactions in decreasing order by amount. If two contributors contain the same transaction amount, sort by alphabetical order of their names.

We will be grading the query string `query_q1b`.  You may modify our template but the result should contain the same information with the same names.

The output should similar to the following:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>committee_id</th>
      <th>name</th>
      <th>transaction_amt</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>C00473918</td>
      <td>SHENKER, SCOTT</td>
      <td>500000</td>
    </tr>
    <tr>
      <th>1</th>
      <td>C00473918</td>
      <td>SHENKER, SCOTT</td>
      <td>200000</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C00473918</td>
      <td>SHENKER, SCOTT</td>
      <td>200000</td>
    </tr>
    <tr>
      <th>3</th>
      <td>C00619411</td>
      <td>SIMONS, NAT</td>
      <td>136300</td>
    </tr>
    <tr>
      <th>4</th>
      <td>C00619411</td>
      <td>SIMONS, NAT</td>
      <td>100200</td>
    </tr>
  </tbody>
</table>



In [ ]:
# complete the query string
query_q1b = """
SELECT 
    ... AS committee_id,
    ... AS name,
    ... AS transaction_amt
FROM ...
WHERE ...
ORDER BY ...
"""

### BEGIN SOLUTION
query_q1b = '''
SELECT 
    cmte_id AS committee_id, 
    name AS name,
    transaction_amt AS transaction_amt
FROM indiv_sample_berkeley
WHERE transaction_amt > 100000
ORDER BY transaction_amt DESC, name
'''
### END SOLUTION
pd.read_sql(query_q1b, engine)

In [ ]:
res = pd.read_sql(query_q1b, engine)
assert res.shape == (5, 3)
assert set(res.columns) == set(['committee_id', 'name', 'transaction_amt'])
assert all(res == res.sort_values('transaction_amt', ascending=False))

### BEGIN HIDDEN TESTS
expected = [['C00473918', 'SHENKER, SCOTT', 500000],
            ['C00473918', 'SHENKER, SCOTT', 200000],
            ['C00473918', 'SHENKER, SCOTT', 200000],
            ['C00619411', 'SIMONS, NAT', 136300],
            ['C00619411', 'SIMONS, NAT', 100200]]
assert set(res['name'].unique()) == set({'SHENKER, SCOTT', 'SIMONS, NAT'})
### END HIDDEN TESTS

### Question 1c

Of course, only looking at individual transactions could be misleading, since each contributor can make several transactions. A more interesting question is: How much did each contributor give *in total*?  Find the total transaction amounts after grouping by the name.  This time, just use `LIMIT 20` to limit your results to the top 20 total donations. Sort the results in descending order by total contribution amount. Break ties by alphabetical order of name as in the previous question.

We will be grading the query string `query_q1c`.  You may modify our template but the result should contain the same information with the same names.

The output should look something like: 


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>name</th>
      <th>total_transaction_amt</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>SHENKER, SCOTT</td>
      <td>1334600</td>
    </tr>
    <tr>
      <th>1</th>
      <td>SIMONS, NAT</td>
      <td>363400</td>
    </tr>
    <tr>
      <th>2</th>
      <td>SIMONS, NATHANIEL</td>
      <td>224700</td>
    </tr>
    <tr>
      <th>3</th>
      <td>WILKINS, MICHAEL IAN</td>
      <td>186681</td>
    </tr>
    <tr>
      <th>4</th>
      <td>HEADLEY, MARK</td>
      <td>161700</td>
    </tr>
    <tr>
      <th>5</th>
      <td>GUND, LOUISE LAIDLAW</td>
      <td>148083</td>
    </tr>
    <tr>
      <th>6</th>
      <td>SLATER, AMY</td>
      <td>126340</td>
    </tr>
    <tr>
      <th>7</th>
      <td>HEADLEY, MARK W.</td>
      <td>115400</td>
    </tr>
    <tr>
      <th>8</th>
      <td>HEADLEY, MARK W</td>
      <td>115200</td>
    </tr>
    <tr>
      <th>9</th>
      <td>BERLEKAMP, ELWYN</td>
      <td>114900</td>
    </tr>
    <tr>
      <th>10</th>
      <td>BERNHARDT, ANTHONY</td>
      <td>105350</td>
    </tr>
    <tr>
      <th>11</th>
      <td>SHENKER, SCOTT M</td>
      <td>105000</td>
    </tr>
    <tr>
      <th>12</th>
      <td>LEVIEN, RAPHAEL LINUS</td>
      <td>100000</td>
    </tr>
    <tr>
      <th>13</th>
      <td>SUTTIE, JILL</td>
      <td>93565</td>
    </tr>
    <tr>
      <th>14</th>
      <td>DUIGNAN, SHEILA MARIE</td>
      <td>85000</td>
    </tr>
    <tr>
      <th>15</th>
      <td>HOLTZMAN, STEVEN</td>
      <td>77800</td>
    </tr>
    <tr>
      <th>16</th>
      <td>PARKER, MICHAEL L.</td>
      <td>75500</td>
    </tr>
    <tr>
      <th>17</th>
      <td>FERGUSON, THEODOSIA</td>
      <td>74055</td>
    </tr>
    <tr>
      <th>18</th>
      <td>PARKER, ANN</td>
      <td>73700</td>
    </tr>
    <tr>
      <th>19</th>
      <td>FIDDLER, JERRY</td>
      <td>70800</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q1c = '''
SELECT 
    ... AS name,
    ... AS total_transaction_amt 
FROM indiv_sample_berkeley 
GROUP BY ... 
ORDER BY ... DESC
LIMIT ...
'''
### BEGIN SOLUTION
query_q1c = '''
SELECT 
    name AS name, 
    sum(transaction_amt) AS total_transaction_amt 
FROM indiv_sample_berkeley 
GROUP BY name
ORDER BY total_transaction_amt DESC, name
LIMIT 20
'''
### END SOLUTION

pd.read_sql(query_q1c, engine)

In [ ]:
res = pd.read_sql(query_q1c,engine)
assert res.shape == (20, 2)
assert all(res['total_transaction_amt'] == res['total_transaction_amt'].sort_values(ascending=False))

## BEGIN HIDDEN TESTS
assert set(res['name']) == set({'BERLEKAMP, ELWYN',
                                'BERNHARDT, ANTHONY',
                                'DUIGNAN, SHEILA MARIE',
                                'FERGUSON, THEODOSIA',
                                'FIDDLER, JERRY',
                                'GUND, LOUISE LAIDLAW',
                                'HEADLEY, MARK',
                                'HEADLEY, MARK W',
                                'HEADLEY, MARK W.',
                                'HOLTZMAN, STEVEN',
                                'LEVIEN, RAPHAEL LINUS',
                                'PARKER, ANN',
                                'PARKER, MICHAEL L.',
                                'SHENKER, SCOTT',
                                'SHENKER, SCOTT M',
                                'SIMONS, NAT',
                                'SIMONS, NATHANIEL',
                                'SLATER, AMY',
                                'SUTTIE, JILL',
                                'WILKINS, MICHAEL IAN'})
## END HIDDEN TESTS

### Question 2a

We might also be interested in examining the distribution of contributors by occupation. Let's first compute how many distinct occupations occur in the dataset. You may need to use the `DISTINCT` keyword to solve the problem.

In [ ]:
query_q2a = '''
SELECT 
    ...
FROM indiv_sample_berkeley 
'''
### BEGIN SOLUTION
query_q2a = '''
SELECT 
    COUNT(distinct occupation)
FROM indiv_sample_berkeley 
'''
### END SOLUTION

pd.read_sql(query_q2a,engine)

In [ ]:
res = pd.read_sql(query_q2a,engine)
assert len(res) == 1
assert res.columns == ['count']

### BEGIN HIDDEN TESTS
assert res["count"][0] == 1656
### END HIDDEN TESTS

### Question 2b 
 Write a SQL query which computes for each occupation the number of transactions in the indiv_sample table. Display the top 20 results in descending order by count. 

We will be grading the query string `query_q2b`.  You may modify our template but the result should contain the same information with the same names.

The output should look something like: 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>NOT EMPLOYED</td>
      <td>13105</td>
    </tr>
    <tr>
      <th>1</th>
      <td>RETIRED</td>
      <td>10880</td>
    </tr>
    <tr>
      <th>2</th>
      <td>None</td>
      <td>2651</td>
    </tr>
    <tr>
      <th>3</th>
      <td>PROFESSOR</td>
      <td>2184</td>
    </tr>
    <tr>
      <th>4</th>
      <td>ATTORNEY</td>
      <td>2116</td>
    </tr>
    <tr>
      <th>5</th>
      <td>UNEMPLOYED</td>
      <td>1066</td>
    </tr>
    <tr>
      <th>6</th>
      <td>NONE</td>
      <td>1010</td>
    </tr>
    <tr>
      <th>7</th>
      <td>TEACHER</td>
      <td>891</td>
    </tr>
    <tr>
      <th>8</th>
      <td>SCIENTIST</td>
      <td>854</td>
    </tr>
    <tr>
      <th>9</th>
      <td>ENGINEER</td>
      <td>764</td>
    </tr>
    <tr>
      <th>10</th>
      <td>PHYSICIAN</td>
      <td>753</td>
    </tr>
    <tr>
      <th>11</th>
      <td>PSYCHOLOGIST</td>
      <td>749</td>
    </tr>
    <tr>
      <th>12</th>
      <td>WRITER</td>
      <td>631</td>
    </tr>
    <tr>
      <th>13</th>
      <td>HOMEMAKER</td>
      <td>619</td>
    </tr>
    <tr>
      <th>14</th>
      <td>RETIRED TEACHER</td>
      <td>581</td>
    </tr>
    <tr>
      <th>15</th>
      <td>CONSULTANT</td>
      <td>549</td>
    </tr>
    <tr>
      <th>16</th>
      <td>SOFTWARE ENGINEER</td>
      <td>547</td>
    </tr>
    <tr>
      <th>17</th>
      <td>LAWYER</td>
      <td>532</td>
    </tr>
    <tr>
      <th>18</th>
      <td>ARTIST</td>
      <td>515</td>
    </tr>
    <tr>
      <th>19</th>
      <td>FACILITATOR, EDUCATOR</td>
      <td>514</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q2b = '''
SELECT 
    ... AS occupation, 
    ... AS count
FROM ...
...
...
LIMIT ...
'''
### BEGIN SOLUTION
query_q2b = '''
SELECT 
    occupation AS occupation, 
    count(*) AS count
FROM indiv_sample_berkeley 
GROUP BY occupation
ORDER BY count DESC
LIMIT 20
'''
### END SOLUTION
pd.read_sql(query_q2b, engine)

In [ ]:
res = pd.read_sql(query_q2b,engine)
assert res.shape == (20, 2)
assert set(res.columns) == set(['occupation', 'count'])
assert all(res == res.sort_values('count', ascending=False))
### BEGIN HIDDEN TESTS
assert set(res['occupation']).issuperset(set({'ARTIST',
                                'ATTORNEY',
                                'CONSULTANT',
                                'ENGINEER',
                                'FACILITATOR, EDUCATOR',
                                'HOMEMAKER',
                                'LAWYER',
                                'NONE',
                                'NOT EMPLOYED',
                                'PHYSICIAN',
                                'PROFESSOR',
                                'PSYCHOLOGIST',
                                'RETIRED',
                                'RETIRED TEACHER',
                                'SCIENTIST',
                                'SOFTWARE ENGINEER',
                                'TEACHER',
                                'UNEMPLOYED',
                                'WRITER'}))
### END HIDDEN TESTS

### Questions 2c
Compute the average transaction amount for each occupation, and sort them in descending order.  
Display the top 10 results.

The output should look something like: 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>SOFTWARE OFFICER</td>
      <td>51350.000000</td>
      <td>2</td>
    </tr>
    <tr>
      <th>1</th>
      <td>CHAIRMAN</td>
      <td>20921.428571</td>
      <td>14</td>
    </tr>
    <tr>
      <th>2</th>
      <td>LOUISE GUND FOUNDATION</td>
      <td>18561.000000</td>
      <td>8</td>
    </tr>
    <tr>
      <th>3</th>
      <td>SCIENCE JOURNALIST</td>
      <td>12933.333333</td>
      <td>3</td>
    </tr>
    <tr>
      <th>4</th>
      <td>FINANCE</td>
      <td>10115.947368</td>
      <td>19</td>
    </tr>
    <tr>
      <th>5</th>
      <td>INVESTOR/BOARD MEMBER</td>
      <td>10000.000000</td>
      <td>1</td>
    </tr>
    <tr>
      <th>6</th>
      <td>INVESTMENT MANGER</td>
      <td>10000.000000</td>
      <td>2</td>
    </tr>
    <tr>
      <th>7</th>
      <td>INVESTMENT MANAGER</td>
      <td>9838.297872</td>
      <td>47</td>
    </tr>
    <tr>
      <th>8</th>
      <td>VENTURE CAPITALIST</td>
      <td>8578.571429</td>
      <td>7</td>
    </tr>
    <tr>
      <th>9</th>
      <td>PHILANTHROPIST</td>
      <td>8566.666667</td>
      <td>9</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q2c = '''
SELECT 
    ... AS occupation, 
    ... AS avg_amount,
    ... AS count
FROM indiv_sample_berkeley 
...
...
LIMIT ...
'''

### BEGIN SOLUTION
query_q2c = '''
SELECT 
    occupation AS occupation, 
    avg(transaction_amt) AS amount,
    count(*) AS count
FROM indiv_sample_berkeley 
GROUP BY occupation 
ORDER BY amount DESC
LIMIT 10
'''

### END SOLUTION
pd.read_sql(query_q2c, engine)

In [ ]:
res = pd.read_sql(query_q2c,engine)
assert res.shape == (10, 3)
assert all(res == res.sort_values('amount', ascending=False))

### BEGIN HIDDEN TESTS
assert set(list(res["amount"].astype(int))) == set([51350, 20921, 18561, 12933, 10115, 10000, 10000, 9838, 8578, 8566])
assert set(list(res["occupation"])) == set(['SOFTWARE OFFICER', 'CHAIRMAN', 'LOUISE GUND FOUNDATION', 'SCIENCE JOURNALIST', 'FINANCE', 'INVESTOR/BOARD MEMBER', 'INVESTMENT MANGER', 'INVESTMENT MANAGER', 'VENTURE CAPITALIST', 'PHILANTHROPIST'])
### END HIDDEN TESTS

### Questions 2d

A few of the above occupations contain 1 or 2 transactions which could lead to unreliable conclusions.  Use a `HAVING` clause to keep only occupations for which we have more than 10 transactions. 

As before, display the top 10 occupations that have the largest average total transaction amount.

Your output should look similar to: 

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>CHAIRMAN</td>
      <td>20921.428571</td>
      <td>14</td>
    </tr>
    <tr>
      <th>1</th>
      <td>FINANCE</td>
      <td>10115.947368</td>
      <td>19</td>
    </tr>
    <tr>
      <th>2</th>
      <td>INVESTMENT MANAGER</td>
      <td>9838.297872</td>
      <td>47</td>
    </tr>
    <tr>
      <th>3</th>
      <td>PRINCIPAL</td>
      <td>7576.500000</td>
      <td>20</td>
    </tr>
    <tr>
      <th>4</th>
      <td>INVESTMENT MANAGEMENT</td>
      <td>3712.820513</td>
      <td>39</td>
    </tr>
    <tr>
      <th>5</th>
      <td>INVESTOR</td>
      <td>2606.171429</td>
      <td>70</td>
    </tr>
    <tr>
      <th>6</th>
      <td>SCIENTIST</td>
      <td>1819.542155</td>
      <td>854</td>
    </tr>
    <tr>
      <th>7</th>
      <td>REAL ESTATE DEVELOPER</td>
      <td>1522.727273</td>
      <td>11</td>
    </tr>
    <tr>
      <th>8</th>
      <td>INVESTOR, BOARD DIRECTOR</td>
      <td>1440.625000</td>
      <td>16</td>
    </tr>
    <tr>
      <th>9</th>
      <td>MATH PROFESSOR</td>
      <td>1387.000000</td>
      <td>11</td>
    </tr>
  </tbody>
</table>


In [ ]:
# complete the query string
query_q2d = '''
SELECT 
    ... AS occupation, 
    ... AS avg_amount,
    ... AS count
FROM indiv_sample_berkeley 
GROUP BY ...
HAVING ...
ORDER BY ...
LIMIT ...
'''

### BEGIN SOLUTION
query_q2d = '''
SELECT 
    occupation AS occupation, 
    avg(transaction_amt) AS amount,
    count(*) AS count
FROM indiv_sample_berkeley 
GROUP BY occupation 
HAVING count(*) > 10
ORDER BY amount DESC
LIMIT 10
'''
### END SOLUTION
pd.read_sql(query_q2d, engine)

In [ ]:
res = pd.read_sql(query_q2d, engine)
assert res.shape == (10, 3)
assert all(res == res.sort_values('amount', ascending=False))

### BEGIN HIDDEN TESTS
assert set(list(res["amount"].astype(int))) == set([20921, 10115, 9838, 7576, 3712, 2606, 1819, 1522, 1440, 1387])
assert set(list(res["occupation"])) == set(['CHAIRMAN', 'FINANCE', 'INVESTMENT MANAGER', 'PRINCIPAL', 'INVESTMENT MANAGEMENT', 'INVESTOR', 'SCIENTIST', 'REAL ESTATE DEVELOPER', 'INVESTOR, BOARD DIRECTOR', 'MATH PROFESSOR'])
### END HIDDEN TESTS

### Questions 2e

Let's break the total contributions down by occupation and individual (grouping on both dimensions at once).  Display the `occupation`, `name`, and total contribution amount from the `indiv_sample_berkeley` table. Display the top 10 results order by the `total_amount`.

The output should look something like: 
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>name</th>
      <th>total_amount</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>SCIENTIST</td>
      <td>SHENKER, SCOTT</td>
      <td>1326500</td>
    </tr>
    <tr>
      <th>1</th>
      <td>INVESTMENT MANAGER</td>
      <td>SIMONS, NAT</td>
      <td>363400</td>
    </tr>
    <tr>
      <th>2</th>
      <td>FINANCE</td>
      <td>WILKINS, MICHAEL IAN</td>
      <td>186681</td>
    </tr>
    <tr>
      <th>3</th>
      <td>LOUISE GUND FOUNDATION</td>
      <td>GUND, LOUISE LAIDLAW</td>
      <td>145788</td>
    </tr>
    <tr>
      <th>4</th>
      <td>PRINCIPAL</td>
      <td>SIMONS, NATHANIEL</td>
      <td>133600</td>
    </tr>
    <tr>
      <th>5</th>
      <td>INVESTMENT MANAGEMENT</td>
      <td>HEADLEY, MARK</td>
      <td>132800</td>
    </tr>
    <tr>
      <th>6</th>
      <td>ATTORNEY</td>
      <td>SLATER, AMY</td>
      <td>124340</td>
    </tr>
    <tr>
      <th>7</th>
      <td>CHAIRMAN</td>
      <td>HEADLEY, MARK W</td>
      <td>115200</td>
    </tr>
    <tr>
      <th>8</th>
      <td>SCIENTIST</td>
      <td>SHENKER, SCOTT M</td>
      <td>105000</td>
    </tr>
    <tr>
      <th>9</th>
      <td>CHAIRMAN</td>
      <td>HEADLEY, MARK W.</td>
      <td>100000</td>
    </tr>
  </tbody>
</table>

In [ ]:
# complete the query string
query_q2e = '''
SELECT 
    ... AS occupation,
    ... AS name,
    ... AS total_amount
FROM indiv_sample_berkeley 
...
...
...
'''

### BEGIN SOLUTION
query_q2e = '''
SELECT 
    occupation AS occupation,
    name AS name,
    sum(transaction_amt) AS total_amount
FROM indiv_sample_berkeley
GROUP BY occupation,name
ORDER BY total_amount DESC
LIMIT 10
'''
### END SOLUTION
pd.read_sql(query_q2e, engine)

In [ ]:
res = pd.read_sql(query_q2e, engine)
assert res.shape == (10, 3)
assert all(res == res.sort_values('total_amount', ascending=False))

### BEGIN HIDDEN TESTS
assert set(list(res["total_amount"].astype(int))) == set([1326500, 363400, 186681, 145788, 133600, 132800, 124340, 115200, 105000, 100000])
assert set(list(res["occupation"])).issuperset(set(['SCIENTIST', 'INVESTMENT MANAGER', 'FINANCE', 'LOUISE GUND FOUNDATION', 'PRINCIPAL', 'INVESTMENT MANAGEMENT', 'ATTORNEY', 'CHAIRMAN', 'SCIENTIST']))
### END HIDDEN TESTS

### Question 2f

Let's now examine the **average** of the **total contribution** for each person registered within an occupation. Note we will need to do this in two steps.  We will first need to compute the total contributions for each user and group and then average the totals across groups.  Use the following tested query template to answer this question. Restrict the final output to occupations that have more than 5 individuals.  Display the top 10 results order the results by the average amount in descending order and break ties with the occupation in ascending order. 

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>occupation</th>
      <th>amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>CHAIRMAN</td>
      <td>36612.500000</td>
      <td>8</td>
    </tr>
    <tr>
      <th>1</th>
      <td>SCIENTIST</td>
      <td>27261.210526</td>
      <td>57</td>
    </tr>
    <tr>
      <th>2</th>
      <td>PRINCIPAL</td>
      <td>25255.000000</td>
      <td>6</td>
    </tr>
    <tr>
      <th>3</th>
      <td>FINANCE</td>
      <td>24025.375000</td>
      <td>8</td>
    </tr>
    <tr>
      <th>4</th>
      <td>INVESTOR</td>
      <td>12162.133333</td>
      <td>15</td>
    </tr>
    <tr>
      <th>5</th>
      <td>PSYCHOANALYST</td>
      <td>5955.000000</td>
      <td>12</td>
    </tr>
    <tr>
      <th>6</th>
      <td>INVESTMENT ADVISOR</td>
      <td>5133.333333</td>
      <td>6</td>
    </tr>
    <tr>
      <th>7</th>
      <td>DOCTOR</td>
      <td>4125.909091</td>
      <td>11</td>
    </tr>
    <tr>
      <th>8</th>
      <td>EDUCATION</td>
      <td>3494.375000</td>
      <td>8</td>
    </tr>
    <tr>
      <th>9</th>
      <td>REAL ESTATE</td>
      <td>3426.050000</td>
      <td>20</td>
    </tr>
  </tbody>
</table>


In [ ]:
# complete the query string
query_q2f = '''
SELECT occupation, avg(total_amount) AS amount, count(*)
FROM (
    SELECT ..., ..., ... AS total_amount
    FROM indiv_sample_berkeley 
    GROUP BY ...
) AS total_contributions
GROUP BY ...
HAVING ...
ORDER BY ...
LIMIT ...
'''
### BEGIN SOLUTION
query_q2f = '''
SELECT occupation, avg(total_amount) AS amount, count(*)
FROM (
    SELECT name, occupation, sum(transaction_amt) AS total_amount
    FROM indiv_sample_berkeley 
    GROUP BY name, occupation
) AS total_contributions
GROUP BY occupation
HAVING count(*) > 5
ORDER BY amount DESC, occupation
LIMIT 10
'''
### END SOLUTION
pd.read_sql(query_q2f, engine)

In [ ]:
res = pd.read_sql(query_q2f,engine)
assert res.shape == (10, 3)
assert all(res == res.sort_values('amount', ascending=False))

### BEGIN HIDDEN TESTS
assert set(list(res["amount"].astype(int))) == set([36612, 27261, 25255, 24025, 12162, 5955, 5133, 4125, 3494, 3426])
assert set(list(res["occupation"])) == set(['CHAIRMAN', 'SCIENTIST', 'PRINCIPAL', 'FINANCE', 'INVESTOR', 'PSYCHOANALYST', 'INVESTMENT ADVISOR', 'DOCTOR', 'EDUCATION', 'REAL ESTATE'])
### END HIDDEN TESTS

### Question 3
Now let's examine the committees that appear in the `indiv_sample_berkeley` table, with the information from the `comm` table. First let's take a look of the `comm` table.

In [ ]:
pd.read_sql("select * from comm limit 5", engine)

### Questions 3a
Let's first groupby the transactions in the `indiv_sample_berkeley` table with `cmte_id` column (committee id), then count how many transactions and how much total amount of contribution there are for each `cmte_id`. Select the top 5 committees order by the transactions count.

Your output should look similar to:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>committee_id</th>
      <th>total_amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>C00401224</td>
      <td>2302741</td>
      <td>27327</td>
    </tr>
    <tr>
      <th>1</th>
      <td>C00575795</td>
      <td>1418867</td>
      <td>8807</td>
    </tr>
    <tr>
      <th>2</th>
      <td>C00577130</td>
      <td>598675</td>
      <td>8185</td>
    </tr>
    <tr>
      <th>3</th>
      <td>C00000935</td>
      <td>851328</td>
      <td>5589</td>
    </tr>
    <tr>
      <th>4</th>
      <td>C00042366</td>
      <td>708910</td>
      <td>2438</td>
    </tr>
  </tbody>
</table>

In [ ]:
query_q3a='''
SELECT
    ... AS committee_id,
    ... AS total_amount,
    ... AS count
FROM indiv_sample_berkeley
...
...
...
'''

### BEGIN SOLUTION
query_q3a='''
SELECT
    cmte_id AS committee_id,
    sum(transaction_amt) AS total_amount,
    count(*) AS count
FROM indiv_sample_berkeley
GROUP BY cmte_id
ORDER BY count DESC
LIMIT 5
'''
### END SOLUTION

pd.read_sql(query_q3a, engine)

In [ ]:
res = pd.read_sql(query_q3a,engine)
assert res.shape == (5, 3)
assert all(res == res.sort_values('count', ascending=False))

### BEGIN HIDDEN TESTS
assert set(res['total_amount'].unique()) == set({598675, 708910, 851328, 1418867, 2302741})
### END HIDDEN TESTS

### Questions 3b
Only showing the committee id doesn't tell us much. Let's also select the committee information by joining with the `comm` table using the `cmite_id` field.

You will extend the query we have from q3a and then perform a `JOIN` with the `comm` table. Select extra committee information such as committee name, party_affiliation, city and state from `comm`. Select the top 10 committees order by transaction count.

Your output should look like the following:

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>cmte_name</th>
      <th>party_affiliation</th>
      <th>city</th>
      <th>state</th>
      <th>total_amount</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>ACTBLUE</td>
      <td>None</td>
      <td>SOMERVILLE</td>
      <td>MA</td>
      <td>2302741</td>
      <td>27327</td>
    </tr>
    <tr>
      <th>1</th>
      <td>HILLARY FOR AMERICA</td>
      <td>DEM</td>
      <td>NEW YORK</td>
      <td>NY</td>
      <td>1418867</td>
      <td>8807</td>
    </tr>
    <tr>
      <th>2</th>
      <td>BERNIE 2016</td>
      <td>DEM</td>
      <td>BURLINGTON</td>
      <td>VT</td>
      <td>598675</td>
      <td>8185</td>
    </tr>
    <tr>
      <th>3</th>
      <td>DCCC</td>
      <td>DEM</td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>851328</td>
      <td>5589</td>
    </tr>
    <tr>
      <th>4</th>
      <td>DSCC</td>
      <td>DEM</td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>708910</td>
      <td>2438</td>
    </tr>
    <tr>
      <th>5</th>
      <td>END CITIZENS UNITED</td>
      <td>None</td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>76742</td>
      <td>2050</td>
    </tr>
    <tr>
      <th>6</th>
      <td>HILLARY VICTORY FUND</td>
      <td>None</td>
      <td>NEW YORK</td>
      <td>NY</td>
      <td>1530057</td>
      <td>1659</td>
    </tr>
    <tr>
      <th>7</th>
      <td>MOVEON.ORG POLITICAL ACTION</td>
      <td>None</td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>134619</td>
      <td>1415</td>
    </tr>
    <tr>
      <th>8</th>
      <td>DNC SERVICES CORP./DEM. NAT'L COMMITTEE</td>
      <td>DEM</td>
      <td>WASHINGTON</td>
      <td>DC</td>
      <td>273294</td>
      <td>954</td>
    </tr>
    <tr>
      <th>9</th>
      <td>CATHERINE CORTEZ MASTO FOR SENATE</td>
      <td>DEM</td>
      <td>LAS VEGAS</td>
      <td>NV</td>
      <td>159707</td>
      <td>815</td>
    </tr>
  </tbody>
</table>

In [ ]:
query_q3b = '''
WITH cmte_count AS
(
  SELECT 
      ... AS cmte_id,
      ... AS total_amount,
      ... AS count
  FROM indiv_sample_berkeley 
  ...
)
SELECT 
    ... AS cmte_name, 
    ... AS party_affiliation,
    ... AS city,
    ... AS state,
    ... AS total_amount,
    ... AS count
FROM comm c ... JOIN cmte_count i ON ...
...
...
'''
### BEGIN SOLUTION
query_q3b = '''
WITH cmte_count AS
(
  SELECT 
      cmte_id AS cmte_id,
      sum(transaction_amt) AS total_amount,
      count(*) AS count
  FROM indiv_sample_berkeley 
  GROUP BY cmte_id 
)
SELECT 
    c.cmte_nm AS cmte_name, 
    c.cmte_pty_affiliation AS party_affiliation,
    c.cmte_city AS city,
    c.cmte_st AS state,
    i.total_amount AS total_amount,
    i.count AS count
FROM comm c RIGHT OUTER JOIN cmte_count i ON c.cmte_id = i.cmte_id
ORDER BY count DESC
LIMIT 10
'''
### END SOLUTION

pd.read_sql(query_q3b, engine)

In [ ]:
res = pd.read_sql(query_q3b,engine)[["cmte_name", "count"]]
assert res.shape == (10, 2)
assert all(res == res.sort_values('count', ascending=False))

### BEGIN HIDDEN TESTS
assert set(res['count'].unique()) == set({815, 954, 1415, 1659, 2050, 2438, 5589, 8185, 8807, 27327})
### END HIDDEN TESTS

### Question 3c
From the results in q3b, it seems most of the party affiliation are DEM. So let's count the actual distribution of the party affiliation. Select all the different committee id from `indiv_sample_berkeley`, joining with the `comm` table. Then group the results by party_affiliation and count how many committees there are for each party_affiliation. Order the results by committee count.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>party_affiliation</th>
      <th>count</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>None</td>
      <td>386</td>
    </tr>
    <tr>
      <th>1</th>
      <td>DEM</td>
      <td>58</td>
    </tr>
    <tr>
      <th>2</th>
      <td>REP</td>
      <td>8</td>
    </tr>
    <tr>
      <th>3</th>
      <td>UNK</td>
      <td>4</td>
    </tr>
    <tr>
      <th>4</th>
      <td>NNE</td>
      <td>2</td>
    </tr>
    <tr>
      <th>5</th>
      <td>GRE</td>
      <td>1</td>
    </tr>
  </tbody>
</table>

In [ ]:
query_q3c = '''
WITH cmte AS
(
  SELECT 
      ... AS cmte_id
  FROM indiv_sample_berkeley 
)
SELECT 
    ... AS party_affiliation,
    ... AS count
FROM ...
...
...
'''
### BEGIN SOLUTION
query_q3c = '''
WITH cmte AS
(
  SELECT 
      DISTINCT cmte_id AS cmte_id
  FROM indiv_sample_berkeley 
)
SELECT 
    c.cmte_pty_affiliation AS party_affiliation,
    count(*) AS count
FROM comm c  RIGHT OUTER JOIN cmte i ON c.cmte_id = i.cmte_id
GROUP BY c.cmte_pty_affiliation
ORDER BY count DESC
'''
### END SOLUTION
pd.read_sql(query_q3c, engine)

In [ ]:
res = pd.read_sql(query_q3c, engine)
assert res.shape == (6, 2)
assert all(res == res.sort_values('count', ascending=False))

### BEGIN HIDDEN TESTS
assert set(res['count'].unique()) == set({1, 2, 4, 8, 58, 386})
### END HIDDEN TESTS